In [ ]:
import basedosdados as bd
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
# Para carregar o dado direto no pandas
# df = bd.read_table(dataset_id='br_ibge_pnadc',
# table_id='dicionario',
# billing_project_id="titanium-acumen-430612-i5")

Tabela de Dicionario de Dados PNAD-C
(https://ftp.ibge.gov.br/Trabalho_e_Rendimento/Pesquisa_Nacional_por_Amostra_de_Domicilios_continua/Anual/Microdados/Trimestre/Documentacao_Geral/Definicao_variaveis_derivadas_PNADC_20200211.pdf)

VD5002- Rendimento (efetivo) domiciliar per capita

VD4020 – Rendimento efetivamente recebido em todos os trabalhos para pessoas de 14 anos ou
mais de idade (apenas para pessoas que receberam em dinheiro, produtos ou mercadorias em
qualquer trabalho)

'Idade',  #V2009 

'Cor',    #V2010

Economicamente ativa/não',  #VD4001

'Sexo',   #V2007

In [ ]:
query = """SELECT ano, sigla_uf, id_pessoa, VD4020 as renda, V2009 as idade, V2010 as cor,
VD4001 as economicamente_ativo, V2007 as sexo
FROM `basedosdados.br_ibge_pnadc.microdados`
WHERE ano = 2023
LIMIT 100
"""

ano = 2023
def collect_pnadc_query(ano):
    return f"""
    SELECT 
      ano, 
      ARRAY_AGG(sigla_uf ORDER BY ano DESC LIMIT 1)[OFFSET(0)] as sigla_uf,
      id_pessoa, 
      AVG(VD4020) as renda, 
      ARRAY_AGG(V2009 ORDER BY ano DESC LIMIT 1)[OFFSET(0)] as idade, 
      ARRAY_AGG(V2010 ORDER BY ano DESC LIMIT 1)[OFFSET(0)] as cor,
      ARRAY_AGG(VD4001 ORDER BY ano DESC LIMIT 1)[OFFSET(0)] as economicamente_ativo, 
      ARRAY_AGG(V2007 ORDER BY ano DESC LIMIT 1)[OFFSET(0)] as sexo
    FROM 
      `basedosdados.br_ibge_pnadc.microdados`
    WHERE 
      ano = {ano}
    GROUP BY 
      id_pessoa, ano
    """

In [ ]:
# anos = range(2012,2024)

# for ano in tqdm(anos):
#     query = collect_pnadc_query(ano)
#     dfyear = bd.read_sql(query=query,billing_project_id="titanium-acumen-430612-i5")
#     dfyear.to_feather(f'microdados_pnadc_{ano}.feather');

In [ ]:
anos = range(2016,2023)

for ano in tqdm(anos):
    query = collect_pnadc_query(ano)
    dfyear = bd.read_sql(query=query,billing_project_id="titanium-acumen-430612-i5")
    dfyear.to_feather(f'microdados_pnadc_{ano}.feather');

In [ ]:
print(1)

In [ ]:
# data = bd.read_table(dataset_id='br_ibge_pnadc', limit=100,
#     table_id='microdados',
#     billing_project_id="titanium-acumen-430612-i5")
# for i in data.columns:
#     print(i)